In [2]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.kernel_ridge import KernelRidge

import time

from catboost import CatBoostRegressor

import re

from statsmodels.stats.outliers_influence import variance_inflation_factor
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVR

# We will write customized regressor classes which inherits the following base classes from sklearn.
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone

import pickle

from dictionaries import *;
from RegressorEncapsulation import *;

pd.set_option('display.max_columns', 500)

In [3]:
def dummify(df, non_dummies, dummies):
    for dummified in dummies:
        for original in non_dummies:
            if original in dummified:
                orig_name = f'{original}_'
                value = dummified.replace(orig_name, '')
                df[dummified] = df[original].map(lambda x: 1 if x == value else 0)
    df=df.drop(non_dummies,axis=1)
    return df

In [4]:
svrg_backend_scaler = StandardScaler()
svr_price_scaler = StandardScaler()

In [41]:
housing_address = pd.read_csv('./house_coordinates_address.csv', index_col = 0);

In [42]:
housing_address.head()

,Address,longitude,latitude
PID,,,
909176150,436 Hayward Ave,-93.651452,42.017780
905476230,3416 West St,-93.664186,42.024697
911128020,320 S 2nd St,-93.614855,42.021389
535377150,1524 Douglas Ave,-93.612065,42.038070
534177230,2304 Fillmore Ave,-93.631893,42.044900


In [44]:
housing_coords = pd.read_csv('../data/ames_housing_price_data_final.csv', index_col = 0);
# housing_address.reset_index(inplace = True)
# housing_address.drop("PID", axis = 1, inplace = True)

# housing_coords.reset_index(inplace = True)
# housing_coords["Address"] = housing_address["Address"]
# housing_coords.set_index("PID", inplace = True)
housing_coords.head()

,SalePrice,Address,GrLivArea,LotArea,OverallQual,BSMT_LowQual,house_age_years,GarageCars,MasVnrType,FullBath,HalfBath,BsmtExposure_ord,SaleTypeNew,Neighborhood,BldgType,PorchSF,BSMT_HighQual,Fireplaces,Pool,BedroomAbvGr,ExterQual,OverallCond,KitchenQual,water_tower,graveyard,police,optician,slipway,bar,cinema,supermarket,hotel,stop,farmyard,christian_catholic,jewish,muslim,garden_centre,christian_lutheran,price_score
PID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
909176150,126000,436 HAYWARD AVE,856,7890,0.428571,856.0,71.210959,2.0,None,1,0,1,0,SWISU,1Fam,166,0.0,1,0,2,0.333333,0.500000,0.333333,2,1,1,1,0,11,0,4,0,0,0,1,0,0,0,3,-0.3
905476230,139500,3416 WEST ST,1049,4235,0.285714,104.0,25.104110,1.0,Brick Face,2,0,2,0,Edwards,TwnhsE,105,945.0,0,0,2,0.666667,0.333333,0.666667,1,1,1,1,0,11,0,4,0,0,0,1,0,1,0,2,0.5
535377150,114000,1524 DOUGLAS AVE,1039,8146,0.142857,405.0,109.402740,1.0,None,1,0,1,0,OldTown,1Fam,279,0.0,0,0,2,0.666667,0.833333,0.333333,0,1,1,1,0,4,1,1,0,0,1,0,0,0,0,3,-0.1
534177230,227000,2304 FILLMORE AVE,1665,8400,0.714286,167.0,8.838356,2.0,None,2,1,1,0,NWAmes,1Fam,45,643.0,0,0,3,0.666667,0.500000,0.666667,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,3,-0.7
908128060,198500,4003 MARIGOLD DR,1922,7301,0.571429,0.0,6.501370,2.0,Brick Face,3,0,0,0,Edwards,1Fam,177,0.0,1,0,4,0.666667,0.333333,0.666667,1,0,0,1,0,2,0,4,3,1,0,0,0,0,0,1,-0.4


In [40]:
housing_coords.to_csv('../data/ames_housing_price_data_v6.csv')

In [5]:
housing_coords = pd.read_csv('../data/ames_housing_price_data_v6.csv', index_col = 0);
housing_coords["Address"] = housing_address["Address"]
# if "SalePrice.1" in housing_coords.columns:
#     housing_coords.drop("SalePrice.1", axis = 1, inplace = True);

y = housing_coords["SalePrice"]
y_std = svr_price_scaler.fit_transform(np.array(np.log10(y)).reshape(-1,1));
front_end = housing_coords.copy();
front_end.reset_index(inplace = True)

In [29]:
housing_coords.head()

,PID,SalePrice,Address,GrLivArea,LotArea,OverallQual,BSMT_LowQual,house_age_years,GarageCars,MasVnrType,FullBath,HalfBath,BsmtExposure_ord,SaleTypeNew,Neighborhood,BldgType,PorchSF,BSMT_HighQual,Fireplaces,Pool,BedroomAbvGr,ExterQual,OverallCond,KitchenQual,water_tower,graveyard,police,optician,slipway,bar,cinema,supermarket,hotel,stop,farmyard,christian_catholic,jewish,muslim,garden_centre,christian_lutheran
0,909176150,126000,436 Hayward Ave,856,7890,0.428571,856.0,71.210959,2.0,None,1,0,1,0,SWISU,1Fam,166,0.0,1,0,2,0.333333,0.500000,0.333333,2,1,1,1,0,11,0,4,0,0,0,1,0,0,0,3
1,905476230,139500,3416 West St,1049,4235,0.285714,104.0,25.104110,1.0,Brick Face,2,0,2,0,Edwards,TwnhsE,105,945.0,0,0,2,0.666667,0.333333,0.666667,1,1,1,1,0,11,0,4,0,0,0,1,0,1,0,2
2,535377150,114000,320 S 2nd St,1039,8146,0.142857,405.0,109.402740,1.0,None,1,0,1,0,OldTown,1Fam,279,0.0,0,0,2,0.666667,0.833333,0.333333,0,1,1,1,0,4,1,1,0,0,1,0,0,0,0,3
3,534177230,227000,1524 Douglas Ave,1665,8400,0.714286,167.0,8.838356,2.0,None,2,1,1,0,NWAmes,1Fam,45,643.0,0,0,3,0.666667,0.500000,0.666667,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,3
4,908128060,198500,2304 Fillmore Ave,1922,7301,0.571429,0.0,6.501370,2.0,Brick Face,3,0,0,0,Edwards,1Fam,177,0.0,1,0,4,0.666667,0.333333,0.666667,1,0,0,1,0,2,0,4,3,1,0,0,0,0,0,1


In [ ]:
#function and dictionaries required to transform front-end to back-end
dummies = [   
    'Neighborhood_Blueste',
    'Neighborhood_BrDale',
    'Neighborhood_BrkSide',
    'Neighborhood_ClearCr',
    'Neighborhood_CollgCr',
    'Neighborhood_Crawfor',
    'Neighborhood_Edwards',
    'Neighborhood_Gilbert', 
    'Neighborhood_Greens', 
    'Neighborhood_GrnHill',
    'Neighborhood_IDOTRR',
    'Neighborhood_Landmrk',
    'Neighborhood_MeadowV',
    'Neighborhood_Mitchel',
    'Neighborhood_NAmes',
    'Neighborhood_NPkVill',
    'Neighborhood_NWAmes',
    'Neighborhood_NoRidge',
    'Neighborhood_NridgHt',
    'Neighborhood_OldTown',
    'Neighborhood_SWISU',
    'Neighborhood_Sawyer',
    'Neighborhood_SawyerW',
    'Neighborhood_Somerst',
    'Neighborhood_StoneBr',
    'Neighborhood_Timber',
    'Neighborhood_Veenker',
    'BldgType_2fmCon',
    'BldgType_Duplex',
    'BldgType_Twnhs',
    'BldgType_TwnhsE',
    'MasVnrType_None',
    'MasVnrType_Stone'
    ]

non_dummies=['Neighborhood', 'BldgType', 'MasVnrType']

In [ ]:
#transformation of front-end to back-end, and catboost application
back_end = front_end.copy()
back_end.drop(columns = ['PID', 'SalePrice'], axis =1, inplace = True)
back_end['ExterQualDisc']=back_end['ExterQual']-back_end['OverallQual']
back_end['OverallCondDisc']=back_end['OverallCond']-back_end['OverallQual']
back_end['KitchenQualDisc']=back_end['KitchenQual']-back_end['OverallQual']
back_end=back_end.drop(['ExterQual','OverallCond','KitchenQual'],axis=1)

back_end = dummify(back_end, non_dummies, dummies)

back_end['GrLivArea_log'] = np.log10(back_end['GrLivArea'])
back_end['LotArea_log'] = np.log10(back_end['LotArea'])
back_end.drop(['GrLivArea', 'LotArea'], axis = 1, inplace = True)

# Standardize
back_end_svr = pd.DataFrame(svrg_backend_scaler.fit_transform(back_end), columns = back_end.columns)


In [ ]:
print(back_end_svr.shape)
print(y_std.shape)
back_end_svr.head()

In [ ]:
svrg = SVR(C= 6000, epsilon = 0.1, gamma = 6e-5, max_iter=-1, shrinking=True);

svrg.train(back_end_svr, y_std)
